# Librery

In [1]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt  # for 畫圖用
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense,  Conv1D, MaxPooling1D, Flatten
from tensorflow.keras import optimizers

2023-12-20 06:44:16.761607: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-20 06:44:16.780733: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 06:44:16.780755: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 06:44:16.780770: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 06:44:16.785285: I tensorflow/core/platform/cpu_feature_g

# Make Dictionary for One-hot Encoding

In [7]:
# Import the training set
train_log = pd.read_csv('/user_data/my/作業/data/train_logs.csv')  # 讀取訓練集
train_log = train_log[["id","down_event"]]
train_log

,id,down_event
0,001519c8,Leftclick
1,001519c8,Leftclick
2,001519c8,Shift
3,001519c8,q
4,001519c8,q
...,...,...
8405893,fff05981,Leftclick
8405894,fff05981,Shift
8405895,fff05981,q
8405896,fff05981,Leftclick


In [8]:
train_score = pd.read_csv('/user_data/my/作業/data/train_scores.csv')  # 讀取訓練集label
train_score = train_score.set_index("id")
train_score

,score
id,
001519c8,3.5
0022f953,3.5
0042269b,6.0
0059420b,2.0
0075873a,4.0
...,...
ffb8c745,3.5
ffbef7e5,4.0
ffccd6fd,1.5


In [10]:
# see all key
D = train_log['down_event'].unique()
D

array(['Leftclick', 'Shift', 'q', 'Space', 'Backspace', '.', ',', 'Enter',
       'ArrowLeft', "'", ';', 'ArrowRight', '-', '?', 'Tab', '"',
       'ArrowUp', 'ArrowDown', 'Rightclick', '=', 'CapsLock', 'Control',
       'c', 'v', '/', 'Delete', ':', 'z', '[', '$', '(', ')', '+', 'Home',
       'End', '\\', 'Meta', '*', '&', 'AudioVolumeMute', 'x', '!',
       'Insert', 'MediaPlayPause', 'NumLock', '%', 'V', '>', 'Alt',
       'AudioVolumeUp', 'ContextMenu', 'AudioVolumeDown', 'a', '<',
       'PageDown', ']', 'Middleclick', '@', 'F12', 'j', '\x96', 'Dead',
       't', 's', 'n', 'y', '{', 'ScrollLock', '¿', 'Process', '}',
       'MediaTrackPrevious', 'MediaTrackNext', 'F3', '^', 'Unidentified',
       'Cancel', '2', 'i', 'd', 'r', 'e', '`', '\x9b', 'm', '#', '~',
       'PageUp', 'T', 'A', 'b', 'S', 'ModeChange', '_', 'Escape', 'F11',
       'Unknownclick', 'AltGraph', 'F10', 'h', 'F15', 'Clear', 'OS', 'F',
       'C', 'o', 'Ä±', 'f', 'u', 'w', 'p', 'g', 'M', 'l', '|',
       'â\x80\x

In [8]:
np.save("key.npy",D)

In [9]:
# one-hot encoding dictionary
D_cat = to_categorical(range(len(D)))
D_ind = { D[i]:D_cat[i] for i in range(len(D)) }
D_ind[0] = np.zeros(len(D))
D_ind

{'Leftclick': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'Shift': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [40]:
def oneHotDict(df_word_list):
    print(f"enter oneHotDict")
    
    unique_list = df_word_list.unique()
    list_len = len(unique_list)
    category_list = to_categorical(range(list_len))
    # index_dict = { unique_list[i]:category_list[i] for i in range(list_len) }
    
    index_dict = {}
    for i in range(list_len):
        index_dict[unique_list[i]] = category_list[i]
        
    index_dict[0] = np.zeros(list_len)
    
    print(f"exit oneHotDict")
    return index_dict

# One-hot Encoding Data

In [16]:
def eventPerEssay(df):
    id_class = df.groupby('id')
    event_per_essay = [ [id, len(id_class.groups[id])] for id in train_log['id'].unique() ]
    event_per_essay = np.array(event_per_essay)
    return id_class, event_per_essay

In [14]:
id_class = train_log.groupby('id')
event_per_essay = [ [id, len(id_class.groups[id])] for id in train_log['id'].unique() ]
event_per_essay = np.array(event_per_essay)
event_per_essay

array([['001519c8', '2557'],
       ['0022f953', '2454'],
       ['0042269b', '4136'],
       ...,
       ['ffccd6fd', '3063'],
       ['ffec5b38', '3242'],
       ['fff05981', '3619']], dtype='<U21')

In [51]:
def oneHot_eachEvent( df:pd.DataFrame, num_desire:int, index_dict:dict )->list:
    # print(f"enter oneHot_eachEvent")
    
    # 確保 r_df 會有 num_desire 筆 "down_event"
    r_df = df[["down_event"]]
    if(len(df)<num_desire):
        # print("enter padding")
        padding = { "down_event":["0"]*(num_desire-len(df)) }
        padding = pd.DataFrame(padding)
        r_df = pd.concat([r_df, padding], axis=0)

    r_df = r_df.reset_index().drop("index", axis=1)

    # 製作 r_log (return value)
    r_log = []
    for i in range(num_desire):
        # print(f"enter event {i}")
        event = r_df.at[i, "down_event"]
        r_log.append(index_dict[event])
        
    # print(f"exit oneHot_eachEvent")
    return r_log

In [54]:
def Doc2Inputs(doc_list:pd.DataFrame, score_list:pd.DataFrame)->(np.ndarray, np.ndarray):
    id_class, event_per_essay = eventPerEssay(doc_list)
    index_dict = oneHotDict(doc_list['down_event'])
    
    augmented_doc_list = []
    augmented_score_list = []
    for ind_essay, num_event in event_per_essay:
        print(f"ind_essay = {ind_essay}")
        cur_df = train_log.loc[id_class.groups[ind_essay]].reset_index().drop("index", axis=1)
        cur_score = score_list.at[ind_essay,"score"]

        passage_size = 1000
        for ind_passage in range(int(num_event)//passage_size + 1):
            # print(f"ind_passage = {ind_passage}")
            sub_cur_df = cur_df[ind_passage*1000:(ind_passage+1)*1000]
            augmented_doc_list.append( oneHot_eachEvent(sub_cur_df, passage_size, index_dict) )
            augmented_score_list.append([cur_score])
    
    augmented_doc_list   = np.array(augmented_doc_list)
    augmented_score_list = np.array(augmented_score_list).reshape(-1)
    
    return augmented_doc_list, augmented_score_list

In [55]:
arr_trainLog_2D_augment, arr_trainScore_2D_augment = Doc2Inputs(train_log, train_score)

enter oneHotDict
exit oneHotDict
ind_essay = 001519c8
ind_essay = 0022f953
ind_essay = 0042269b
ind_essay = 0059420b
ind_essay = 0075873a
ind_essay = 0081af50
ind_essay = 0093f095
ind_essay = 009e23ab
ind_essay = 00e048f1
ind_essay = 00e1f05a
ind_essay = 00e713bd
ind_essay = 00f0737e
ind_essay = 00f8e84c
ind_essay = 00fc9a6a
ind_essay = 0144e4d5
ind_essay = 014e7ae9
ind_essay = 015aa732
ind_essay = 0178a105
ind_essay = 0182aa1c
ind_essay = 0190ff4c
ind_essay = 01963e20
ind_essay = 019737b6
ind_essay = 01992d32
ind_essay = 01c359fc
ind_essay = 01d0ba4b
ind_essay = 01d602a7
ind_essay = 0249a095
ind_essay = 0262bf61
ind_essay = 026be946
ind_essay = 0294b4f5
ind_essay = 02a41d1a
ind_essay = 02cf6a52
ind_essay = 02d3c9fc
ind_essay = 02e86b6a
ind_essay = 031c0c58
ind_essay = 034d61db
ind_essay = 035f09fc
ind_essay = 036cfd42
ind_essay = 0395b217
ind_essay = 03971ddf
ind_essay = 040c429b
ind_essay = 0417d421
ind_essay = 0432f117
ind_essay = 0445b534
ind_essay = 044b274d
ind_essay = 044c5c54
i

In [58]:
arr_trainScore_2D_augment.shape

(9638,)

In [57]:
arr_trainLog_2D_augment.shape

(9638, 1000, 131)

In [10]:
# trainLog_2D_augment = []
# trainScore_2D_augment = []
# for ind, [ind_essay, num_event] in enumerate(event_per_essay):
#   # print(ind_essay,num_event)
#   cur_df = train_log.loc[id_class.groups[ind_essay]].reset_index().drop("index", axis=1)
#   cur_score = train_score.at[ind_essay,"score"]

#   for ind_event_thousand in range(int(num_event)//1000 + 1):
#     # print(f"ind_event_thousand = {ind_event_thousand}")
#     sub_cur_df = cur_df[ind_event_thousand*1000:(ind_event_thousand+1)*1000]
#     trainLog_2D_augment.append( oneHot_eachEvent(sub_cur_df, 1000) )
#     trainScore_2D_augment.append([cur_score])

001519c8 2557
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0022f953 2454
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0042269b 4136
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
ind_event_thousand = 3
ind_event_thousand = 4
enter padding
0059420b 1556
ind_event_thousand = 0
ind_event_thousand = 1
enter padding
0075873a 2531
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0081af50 2211
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0093f095 1765
ind_event_thousand = 0
ind_event_thousand = 1
enter padding
009e23ab 2353
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
00e048f1 1585
ind_event_thousand = 0
ind_event_thousand = 1
enter padding
00e1f05a 7826
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
ind_event_thousand = 3
ind_event_thousand = 4
ind_event_thousand = 

In [18]:
arr_trainScore_2D_augment = np.array(trainScore_2D_augment)
arr_trainScore_2D_augment = arr_trainScore_2D_augment.reshape(-1)
arr_trainScore_2D_augment

array([3.5, 3.5, 3.5, ..., 4. , 4. , 4. ])

In [21]:
arr_trainLog_2D_augment = np.array(trainLog_2D_augment)
arr_trainLog_2D_augment = arr_trainLog_2D_augment.reshape(9638, 1000, 131,1)
arr_trainLog_2D_augment.shape

(9638, 1000, 131, 1)

In [23]:
np.save("/user_data/my/作業/data/trainScore_2D_augment.npy", arr_trainScore_2D_augment)

In [24]:
np.save("/user_data/my/作業/data/trainLog_2D_augment.npy", arr_trainLog_2D_augment)

# Building Model

In [11]:
trainScore_2D_augment = np.load("/user_data/my/作業/data/trainScore_2D_augment.npy")
trainLog_2D_augment   = np.load("/user_data/my/作業/data/trainLog_2D_augment.npy")

In [75]:
print(trainLog_2D_augment.shape)
print(trainScore_2D_augment.shape)

(9638, 1000, 131, 1)
(9638,)


In [108]:
model1D = Sequential(
    [
        Conv1D(10, 3, padding="valid", activation='linear', name = "conv_find"),
        MaxPooling1D(3, padding="valid", strides=1, name = "pool_find"),
        Conv1D( 5, 3, padding="valid", activation='linear', name = "conv_pattern"),
        MaxPooling1D(10, padding="valid", name="pool_pattern"),
        Conv1D( 5, 3, padding="valid", activation='linear', name = "conv_contigious"),
        MaxPooling1D(10, padding="valid", name="pool_contigious"),
        Flatten(name="flatten"),
        Dense(10, activation='linear', name="dense_nn"),
        BatchNormalization(name = "BatchNormalization"),
        Dense(1, activation='linear', name="dense_scoring")
    ],
    "shortCut_vs_score_1D"
)

In [109]:
input_shape = (None, 1000, 131)
model1D.build(input_shape)
model1D.summary()

Model: "shortCut_vs_score_1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_find (Conv1D)          (None, 998, 10)           3940      
                                                                 
 pool_find (MaxPooling1D)    (None, 996, 10)           0         
                                                                 
 conv_pattern (Conv1D)       (None, 994, 5)            155       
                                                                 
 pool_pattern (MaxPooling1D  (None, 99, 5)             0         
 )                                                               
                                                                 
 conv_contigious (Conv1D)    (None, 97, 5)             80        
                                                                 
 pool_contigious (MaxPoolin  (None, 9, 5)              0         
 g1D)                                         

In [110]:
model1D.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mean_squared_error']
              )

In [111]:
train_feature = trainLog_2D_augment.reshape((9638, 1000, 131))
train_feature.shape

(9638, 1000, 131)